<a href="https://colab.research.google.com/github/AkashKoley012/Computer-Vision-Projects/blob/main/Face%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!kaggle datasets download -d andrewmvd/face-mask-detection

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/face-mask-detection
License(s): CC0-1.0
 99% 394M/398M [00:08<00:00, 75.4MB/s]
100% 398M/398M [00:08<00:00, 50.3MB/s]


In [4]:
import zipfile
with zipfile.ZipFile('/content/face-mask-detection.zip', 'r') as zip_ref:
    zip_ref.extractall('face-mask')

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
from tqdm import tqdm
import torch
import torch.nn as nn

In [2]:
os.remove('/content/face-mask-detection.zip')

IsADirectoryError: [Errno 21] Is a directory: '/content/face-detection'

In [18]:
' '.join(['a','b'])

'a b'

In [35]:
def createData(dir):

  img = []
  cat = []
  loc = []

  for file in tqdm(os.listdir(f'/content/face-detection/images/{dir}')):
    image = cv2.imread(f'/content/face-detection/images/{dir}/{file}')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img.append(image)
    with open(f'/content/face-detection/labels2/{file[:-3]+"txt"}', 'r') as f:
      lines = f.read()

    for line in lines.split('\n'):
      if line == '':
        continue
      line = line.split(' ')
      loc.append(tuple(line[-4:]))
      cat.append(' '.join(line[:-4]))

  df = pd.DataFrame({'image': img, 'category': cat, 'location': loc})
  return df

In [ ]:
train_df = createData(dir='train')

 31%|███       | 4133/13386 [01:21<1:10:49,  2.18it/s]

In [32]:
train_df.head()

,image,category,location
0,"[[[183, 186, 155], [184, 187, 156], [185, 188,...",Human face,"(387.68320800000004, 20.48, 569.369592, 248.32)"
